# Mixer correction

Setup
-----

First, we are going to import the required packages.

In [3]:
# Import ipython widgets
import json
import math
import os

import ipywidgets as widgets
import matplotlib.pyplot
import numpy as np
import rich
import pandas as pd

# Set up the environment.
import scipy.signal
from IPython.display import display
from ipywidgets import fixed, interact, interact_manual, interactive

from qblox_instruments import Cluster, PlugAndPlay
# Set notebook width to 100%
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# This is the table that we want to populate
calibration = pd.DataFrame()
calibration["module"]  = []
calibration["complex_output"] = []
calibration["sequencer index"] = []
calibration["lo_freq (Hz)"] = []
calibration["if (Hz)"] = []
calibration["dc_mixer_offset_I"] = []
calibration["dc_mixer_offset_Q"] = []
calibration["mixer_amp_ratio"] = []
calibration["mixer_phase_error_deg"] = []
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg


## Create Q1ASM program to play on the device

In [5]:
#Waveform dictionary (data will hold the samples and index will be used to select the waveforms in the instrument).
waveforms = {
             "I":     {"data": [], "index": 0},
             "Q":     {"data": [], "index": 1}
            }

I_amplitude = 0.2
waveforms["I"]["data"] = [I_amplitude]*8000
waveforms["Q"]["data"] = [0.0]*8000

#Sequence program.
prog = f"start: set_mrk 15 \n play 0,1,1000\n set_mrk 15 \n play 0,1,1000\njmp @start"

#Sequence program which stops.
# prog = """
#         move 1250000, R0
#         set_mrk    15
# start:
#         reset_ph
#         upd_param  4
#         play       0,1,4
#         wait       7996
#         loop       R0,@start
#         set_mrk    0
#         upd_param  4
#         stop
# """

#Reformat waveforms to lists if necessary.
for name in waveforms:
    if str(type(waveforms[name]["data"]).__name__) == "ndarray":
        waveforms[name]["data"] = waveforms[name]["data"].tolist()  # JSON only supports lists

#Add sequence program and waveforms to single dictionary and write to JSON file.
wave_and_prog_dict = {"waveforms": waveforms, "weights": {}, "acquisitions": {}, "program": prog}
with open("sequence.json", 'w', encoding='utf-8') as file:
    json.dump(wave_and_prog_dict, file, indent=4)
    file.close()

## Select Device

We scan for the available devices connected via ethernet using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/api_reference/pnp.html) for more info).

In [6]:
# Scan for available devices and display
with PlugAndPlay() as p:
    # get info of all devices
    device_list = p.list_devices()
    device_keys = list(device_list.keys())

# rich.print(device_list)
# create widget for names and ip addresses
connect = widgets.Dropdown(
    options=[(device_list[key]["identity"]["ip"]) for key in device_list.keys()],
    description="Select Device",
)
display(connect)

Dropdown(description='Select Device', options=('192.0.2.72', '192.0.2.142', '192.0.2.143', '192.0.2.141'), val…

In [7]:
# close all previous connections to the cluster
Cluster.close_all()

#device_name = "pingu_cluster"
cluster_name = "loki_cluster"
# ip_address = connect.v./alue
ip_address = '192.0.2.141'
# ip_address = '192.0.2.142'
# connect to the cluster and reset
cluster = Cluster(cluster_name, ip_address)
cluster.reset()
print(f"{cluster_name} connected at {ip_address}")
cluster.identify()

loki_cluster connected at 192.0.2.141


## Select Module in Device

In [82]:
# Find all QRM/QCM modules
cluster.reset()
available_slots = {}
for module in cluster.modules:
    # if module is currently present in stack
    if cluster._get_modules_present(module.slot_idx):
        # check if QxM is RF or baseband
        if module.is_rf_type:
            available_slots[f"module{module.slot_idx}"] = ["QCM-RF", "QRM-RF"][
                module.is_qrm_type
            ]
        else:
            available_slots[f"module{module.slot_idx}"] = ["QCM", "QRM"][
                module.is_qrm_type
            ]

# List of all QxM modules present
connect_qxm = widgets.Dropdown(options=[key for key in available_slots.keys()])
display(connect_qxm)

Dropdown(options=('module1', 'module2', 'module3', 'module4', 'module5', 'module6', 'module7', 'module8', 'mod…

In [83]:
# Connect to the cluster QxM module
module = connect_qxm.value
print(module)
qxm = getattr(cluster, module)
print(f"{available_slots[connect_qxm.value]} connected")
print(cluster.get_system_status())

module16
QRM-RF connected
Status: OKAY, Flags: NONE, Slot flags: NONE


In [84]:
# qxm.out1_att(30) is this needed for QCMRF , QRMRF without demodulator?

## Select complex output

In [85]:
outputs = set(map(lambda i: i.split("_")[0], filter(lambda k: "out" in k and (not k.startswith("_")), dir(qxm))))
select_out = widgets.Dropdown(options = [key for key in outputs])
display(select_out)

Dropdown(options=('disconnect', 'out0'), value='disconnect')

In [96]:
ReadOut_target_frequencies = {
    "q06": 6832073301.189378,
    "q07": 7080671753.192583,
    "q08": 6397230987.533971,
    "q09": 7276184309.188224,
    "q10": 7252681537.885626,
}
average_RO_freq = np.mean(list(ReadOut_target_frequencies.values()))
LO_ReadOut = average_RO_freq - 100e6
print(f'{average_RO_freq = }')
# qb_freq = np.array([3.69374	,3.34177	,3.5669	,3.30677	,3.85258	,3.16559	,3.90661	,3.22831	,3.89478	,3.31304,3.74806	,3.30409	,3.88604	,3.25294	,3.98161
# ])*1e9
# lo = qb_freq + 100e6
# lo_list = [print(i) for i in enumerate(lo)]

average_RO_freq = np.float64(6967768377.797956)


In [97]:
lo_frequency = LO_ReadOut

In [98]:
# lo_frequency = float(input(f"Enter desired LO frequency (Hz) for '{select_out.value}': "))
# lo_frequency = 3.5e9
if qxm.module_type.name == "QRM":
    # qrms only have one complex output
    qxm.out0_in0_lo_en(True)
    qxm.out0_in0_lo_freq(lo_frequency)
else:
    lo_en = getattr(qxm, f"{select_out.value}_lo_en")
    lo_freq = getattr(qxm, f"{select_out.value}_lo_freq")
    lo_en(True)
    lo_freq(lo_frequency)

## Select Sequencer in Module

In [99]:
qxm.stop_sequencer()
sequencers = { k : getattr(qxm, k) for k in filter(lambda a: a.startswith("sequencer") and (a != "sequencers"), dir(qxm)) }
select_seq = widgets.Dropdown(options = [key for key in sequencers.keys()])
display(select_seq)

Dropdown(options=('sequencer0', 'sequencer1', 'sequencer2', 'sequencer3', 'sequencer4', 'sequencer5'), value='…

## Upload program to sequencer

In [126]:
seq = sequencers[select_seq.value]
qubit = 'q10'
MY_IF = ReadOut_target_frequencies[qubit] - LO_ReadOut
seq.nco_freq(MY_IF)
print(f'{MY_IF = }')
# seq.nco_freq(MY_IF := float(input("ENTER IF: ")))   # using 10 MHz IF

MY_IF = np.float64(384913160.0876694)


In [128]:
seq.sequence(os.path.join(os.getcwd(), "sequence.json"))
seq.mod_en_awg(True)
print(f'{MY_IF / 1e6} MHz' )
print(seq.name)

384.9131600876694 MHz
loki_cluster_module16_sequencer0


## Control sliders

In [129]:
seq.connect_out0('IQ')
# The following does not apply for QRMRF modules
# seq.connect_out1('off')
# The following two apply to baseband modules not RF modules
# seq.connect_out2('off')
# seq.connect_out3('off')

seq.gain_awg_path0(1)
seq.gain_awg_path1(1)

In [130]:
# Expect to see the tones on the spectrum analyzer when this cell is run
qxm.arm_sequencer(seq.seq_idx)
qxm.start_sequencer(seq.seq_idx)

In [131]:
rich.print(f"1.\tConnect complex output '{select_out.value}' of cluster '{cluster_name}' module '{module}' to a spectrum analyser.")
rich.print(f"2.\tIn the spectrum analyser, set the center frequency to {lo_frequency/1e9} GHz")
rich.print(f"3.\tAdjust the gain ratio and phase offset sliders until the LSB is gone.")
rich.print(f"4.\tAdjust the I and Q offsets until the LO peak dissapears.")
rich.print(f"5.\tWhen only RSB is visible in the spectrum analyser, save the calibration with the last cell.")

calibrated_values = [ - 3.0245, - 5.3719,0.999,0] if seq.seq_idx == 0 else calibration.loc[len(calibration)-1][-4:].values.tolist() # storage array
#calibrated_values = [-0.004462799999999999*1000, -0.0097599*1000, 0.9824, -5.3845]

def set_offset_I(offset_I):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path0(offset_I) # qrm have only one output
    # elif qxm.module_type.name == "QCM":
    #     seq.offset_awg_path0(offset_I)
    else:
        #print(f"{select_out.value}_offset_path0")
        offset_path0 = getattr(qxm, f"{select_out.value}_offset_path0")
        offset_path0(offset_I)

    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[0] = offset_I

def set_offset_Q(offset_Q):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path1(offset_Q) # qrm have only one output
    # elif qxm.module_type.name == "QCM":
    #     seq.offset_awg_path1(offset_Q)
    else:
        offset_path1 = getattr(qxm, f"{select_out.value}_offset_path1")
        offset_path1(offset_Q)
        
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[1] = offset_Q

def set_gain_ratio(gain_ratio):
    seq.mixer_corr_gain_ratio(gain_ratio)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[2] = gain_ratio

def set_phase_offset(phase_offset):
    seq.mixer_corr_phase_offset_degree(phase_offset)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[3] = phase_offset

interact(
    set_offset_I, offset_I=widgets.FloatSlider(min=-20.0, max=20.0, step=0.0001, value=calibrated_values[0], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_offset_Q, offset_Q=widgets.FloatSlider(min=-20.0, max=20.0, step=0.0001, value=calibrated_values[1], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_gain_ratio,
    gain_ratio=widgets.FloatSlider(min=0.5, max=2.0, step=0.0001, value=calibrated_values[2], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)
interact(
    set_phase_offset,
    phase_offset=widgets.FloatSlider(min=-45.0, max=45.0, step=0.00001, value=calibrated_values[3], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)

1.      Connect complex output 'out0' of cluster 'loki_cluster' module 'module16' to a spectrum analyser.

2.      In the spectrum analyser, set the center frequency to 6.867768377797956 GHz

3.      Adjust the gain ratio and phase offset sliders until the LSB is gone.

4.      Adjust the I and Q offsets until the LO peak dissapears.

5.      When only RSB is visible in the spectrum analyser, save the calibration with the last cell.

interactive(children=(FloatSlider(value=-3.0245, description='offset_I', layout=Layout(width='95%'), max=20.0,…

interactive(children=(FloatSlider(value=-5.3719, description='offset_Q', layout=Layout(width='95%'), max=20.0,…

interactive(children=(FloatSlider(value=0.999, description='gain_ratio', layout=Layout(width='95%'), max=2.0, …

interactive(children=(FloatSlider(value=0.0, description='phase_offset', layout=Layout(width='95%'), max=45.0,…

<function __main__.set_phase_offset(phase_offset)>

In [132]:
calibration.loc[len(calibration)] = [module, select_out.value, seq.seq_idx, lo_frequency, MY_IF, *tuple(calibrated_values)]
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg
0,module16,out0,0,6.967768e+09,-1.356951e+08,-4.6947,-4.4690,0.9994,10.76628
1,module16,out0,0,6.967768e+09,1.129034e+08,-4.6947,-4.4690,1.0362,11.47726
2,module16,out0,0,6.867768e+09,-3.569508e+07,-3.0245,-5.3719,1.0125,11.68040
3,module16,out0,0,6.867768e+09,2.129034e+08,-3.0245,-5.3719,0.9787,11.88353
4,module16,out0,0,6.867768e+09,-4.705374e+08,-3.0245,-5.3719,0.9990,13.00079
5,module16,out0,0,6.867768e+09,4.084159e+08,-3.0245,-5.3719,1.0193,9.54746
6,module16,out0,0,6.867768e+09,3.849132e+08,-3.0245,-5.3719,1.0159,9.14118


In [35]:
calibration.to_csv("mixercorrectionLokeA25QFL.csv")

In [69]:
cluster.reset()